In [1]:
# -*- coding: utf-8 -*-
import os
import pandas as pd
import numpy as np
from PIL import Image
import gc, math
import pickle

from keras.models import Sequential
from keras.optimizers import SGD
from keras.optimizers import Adam
from keras.utils import np_utils
from keras.models import model_from_json
from keras.models import Model
from keras.layers import Input, Dense, Convolution2D, MaxPooling2D, AveragePooling2D, ZeroPadding2D, Dropout, Flatten, merge, Reshape, Activation
from keras.layers.advanced_activations import LeakyReLU, PReLU
from keras.layers.normalization import BatchNormalization
from keras import regularizers
from keras import backend as K
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator

from sklearn.metrics import log_loss, accuracy_score, confusion_matrix

from cnnmodels import vgg_std16_model, preprocess_input, create_rect5, load_img, train_generator, test_generator
from cnnmodels import identity_block, testcv_generator, conv_block, resnet50_model

Using Theano backend.
Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


In [2]:
# Params
img_rows, img_cols = 224, 224 # Resolution of inputs
channel = 3
num_class = 2
ROWS, COLS = 224, 224
BATCHSIZE = 64
SEAL_CLASSES = ['NoS', 'seal']
nb_perClass = int(BATCHSIZE / len(SEAL_CLASSES))
TRAIN_DIR = '../darknet/seals/JPEGImagesBlk'
TEST_DIR = '../darknet/seals/JPEGImagesTest'


In [3]:
# Data generator
train_datagen = ImageDataGenerator(
    horizontal_flip=True,
    vertical_flip=True)

# Fine-tune Example
rfcnCV = pd.read_pickle('../coords/rfcnCV.pkl')
rfcnCV.head(3)

,img,proba,x0,y0,x1,y1,seal
0,228_54,0.894,364.95,295.10,464.95,395.10,0
1,228_54,0.893,228.20,376.30,328.20,476.30,0
2,228_54,0.720,407.75,409.95,507.75,509.95,0


In [4]:
# Lets make our validation set
folds = [rfcnCV.img.str.split('_').apply(lambda x: x[0]).astype(int) % 2 != 0,
        rfcnCV.img.str.split('_').apply(lambda x: x[0]).astype(int) % 2 == 0]
rfcnCV.shape

(146060, 7)

In [28]:
for fold in range(2):
    print "Fold: " + str(fold+1)
    train_df = rfcnCV[~folds[fold]]
    test_df = rfcnCV[folds[fold]]
    valid_df = test_df[:1500]
    
    # validation_data (valid_x,valid_y)
    print "Part ... A"
    df_1 = valid_df
    l = valid_df.groupby('seal').size()
    nb_NoF_valid = math.ceil(l.sum()/10)
    valid_x = np.zeros((valid_df.shape[0], ROWS, COLS, 3), dtype=K.floatx())
    valid_y = np.zeros((valid_df.shape[0], len(SEAL_CLASSES)), dtype=K.floatx())
    i = 0
    for index,row in valid_df.iterrows():
        if index % 300 == 0 : print index
        row = row.tolist()
        image_file = os.path.join(TRAIN_DIR, row[0])
        seal = row[6]
        bbox = row[2:6]
        cropped = load_img(image_file+'.jpg',bbox,target_size=(ROWS,COLS))
        x = np.asarray(cropped, dtype=K.floatx())
        x = preprocess_input(x)
        valid_x[i] = x
        valid_y[i,seal] = 1
        i += 1
    valid_x = valid_x.transpose(0, 3, 1, 2)
    valid_x.shape

    # Load our model
    print "Part ... B"
    nb_epoch = 2
    samples_per_epoch = 50000
    model = vgg_std16_model(ROWS, COLS, channel, num_class)

    # Start Fine-tuning
    print "Part ... C"
    model.fit_generator(train_generator(train_datagen, train_df),
              nb_epoch=nb_epoch,
              samples_per_epoch=samples_per_epoch, #50000,
              verbose=1,
              validation_data=(valid_x, valid_y),
              )

    for layer in model.layers[10:]:
        layer.trainable = True
    model.optimizer.lr = 1e-4
    nb_epoch = 3
    print "Part ... D"
    model.fit_generator(train_generator(train_datagen, df=train_df),
              nb_epoch=nb_epoch,
              samples_per_epoch=samples_per_epoch,
              verbose=1,
              validation_data=(valid_x, valid_y),
              )

    # Test preds save
    print "Part ... E"
    test_preds = model.predict_generator(testcv_generator(test_df), val_samples=test_df.shape[0])
    df = pd.concat([test_df.reset_index(drop=True), pd.DataFrame(test_preds,  columns=['predNoSeal', 'predSeal'])], axis=1)
    df.to_pickle('../coords/vggCVPreds2604_fold' + str(fold+1) + '.pkl')
    df[['img', 'predSeal']].to_csv('../coords/vggCVPreds2604_fold' + str(fold+1) + '.csv', index=False)
    
    # Clean up
    del model, train_df, test_df, valid_df, valid_x, df_1
    gc.collect()
    gc.collect()

Fold: 2
Part ... A
0
300
600
900
1200
Part ... B
Part ... C
Epoch 1/2
50048/50000 [==============================] - 2653s - loss: 0.6821 - acc: 0.6624 - val_loss: 0.2907 - val_acc: 0.8893
Epoch 2/2
50048/50000 [==============================] - 2690s - loss: 0.2089 - acc: 0.9212 - val_loss: 0.2277 - val_acc: 0.9067
Part ... D
Epoch 1/3
50048/50000 [==============================] - 2694s - loss: 0.1553 - acc: 0.9434 - val_loss: 0.1420 - val_acc: 0.9420
Epoch 2/3
50048/50000 [==============================] - 2693s - loss: 0.1329 - acc: 0.9526 - val_loss: 0.1227 - val_acc: 0.9540
Epoch 3/3
50048/50000 [==============================] - 2694s - loss: 0.1134 - acc: 0.9589 - val_loss: 0.1185 - val_acc: 0.9507
Part ... E
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
0
